In [1]:
from allennlp_models.pretrained import load_predictor
from allennlp_models.pretrained import get_pretrained_models

import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

from checklist.pred_wrapper import PredictorWrapper

import pandas as pd

In [2]:
## choose amount of samples (arg-mnr has 4 formats per nsample, arg1 has 1, so it is multiplied by 4 for arg1)
nsamples=25

In [3]:
## list taken from: https://media.bloomsbury.com/rep/files/chapter-1-project-1-5_a-list_of_inanimate_objects.pdf

string_objects = '''
sofa
coffee table
dresser
wooden chair
plush chair
rolling office chair
footstool
kitchen table
toilet
bathtub
bathroom sink
ornate mirror
stove
refrigerator
soap dispenser
tape dispenser
stapler
toothbrush
toothpaste
umbrella
shoe
hammer
pliers
iron
spray bottle
baby bottle
toothbrush
toothpaste
clothespin
teapot
teacup
coffeepot
coffee mug
orange juice box
milk jug
spoon
fork
knife
power outlet
book
briefcase
telephone
desk lamp
standing lamp'''
list_objects = string_objects.split('\n')
list_objects.remove(list_objects[0])

In [4]:
def make_data_pp_attachment(amount_per_format):
    editor = Editor()
    list_pp_attachment = []
    editor = Editor()
    
    test_data = editor.template("I saw the {list_objects} with my eyes.",
                                list_objects = list_objects,
                                meta=True, seed=42,  
                                nsamples=amount_per_format,
                                remove_duplicates=True)
    list_pp_attachment.append(test_data['data'])

    test_data = editor.template("I smelled the {list_objects} with my nose.", 
                                list_objects = list_objects,
                                meta=True, seed=42,  
                                nsamples=amount_per_format,
                                remove_duplicates=True)
    list_pp_attachment.append(test_data['data'])

    test_data = editor.template("I heard the {list_objects} with my ears.",
                                list_objects = list_objects,
                                meta=True, seed=42,  
                                nsamples=amount_per_format,
                                remove_duplicates=True)
    list_pp_attachment.append(test_data['data'])

    test_data = editor.template("I tasted the {list_objects} with my tongue.", 
                                list_objects = list_objects,
                                meta=True, seed=42,  
                                nsamples=amount_per_format,
                                remove_duplicates=True)
    list_pp_attachment.append(test_data['data'])
    list_pp_attachment = [x for xs in list_pp_attachment for x in xs]
    return list_pp_attachment

def get_pp_tags(sentence):
    output = srl_predictor.predict(sentence)
    pp_tags = output['verbs'][0]['tags']
    return pp_tags

def get_pp_descr(sentence):
    output = srl_predictor.predict(sentence)
    pp_descr = output['verbs'][0]['description']
    return pp_descr

def find_final_min_one_arg(list_of_arg):
    return list_of_arg[-2]   

def make_data_pp_attachment_arg1(amount_per_format):
    list_pp_attachment = []

    editor = Editor()
    test_data = editor.template("I saw the man with the {mask}.",
                                list_objects = list_objects,
                                meta=True, seed=42,  
                                nsamples=amount_per_format*4,
                                remove_duplicates=True)
    list_pp_attachment.append(test_data['data'])
    list_pp_attachment = [x for xs in list_pp_attachment for x in xs]
    return list_pp_attachment

def find_final_min_one_arg(list_of_arg):
    return list_of_arg[-2]

In [5]:
list_pp_attachment = make_data_pp_attachment(nsamples)
df_pp = pd.DataFrame(list_pp_attachment)

df_pp[0].to_csv("../datasets/pp_attachment/pp_attachment_arg-mnr.txt", header=None, index=False)

In [6]:
srl_predictor = load_predictor('structured-prediction-srl-bert')

df_pp['pp_tag'] = df_pp[0].apply(get_pp_tags)
df_pp['pp_descr'] = df_pp[0].apply(get_pp_descr)
df_pp[1] = df_pp['pp_tag'].apply(find_final_min_one_arg)

df_pp.drop_duplicates(subset=[0],inplace=True)

lerc is not a registered model.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
df_pp.to_csv("../results/pp_attachment/pp_attachment_arg-mnr_bert.csv")

In [8]:
error_rate_arg_mnr_bert= len(df_pp[df_pp[1]!='I-ARGM-MNR'])/len(df_pp)*100

In [9]:
list_pp_attachment_arg1 = make_data_pp_attachment_arg1(nsamples)

df_pp_arg1 = pd.DataFrame(list_pp_attachment_arg1)
df_pp_arg1['pp_tag'] = df_pp_arg1[0].apply(get_pp_tags)
df_pp_arg1['pp_descr'] = df_pp_arg1[0].apply(get_pp_descr)

df_pp_arg1[1] = df_pp_arg1['pp_tag'].apply(find_final_min_one_arg)

/Users/syb/opt/anaconda3/envs/the/lib/python3.9/site-packages/checklist/text_generation.py:171: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  to_pred = torch.tensor(to_pred, device=self.device).to(torch.int64)


In [10]:
error_rate_arg1_bert = len(df_pp_arg1[df_pp_arg1[1] != 'I-ARG1'])/len(df_pp_arg1)
df_pp_arg1.to_csv("../results/pp_attachment/pp_attachment_arg1_bert.csv")

In [11]:
srl_predictor = load_predictor('structured-prediction-srl')

df_pp['pp_tag'] = df_pp[0].apply(get_pp_tags)
df_pp['pp_descr'] = df_pp[0].apply(get_pp_descr)
df_pp[1] = df_pp['pp_tag'].apply(find_final_min_one_arg)
df_pp.drop_duplicates(subset=[0],inplace=True) 

lerc is not a registered model.


In [12]:
error_rate_arg_mnr_bilstm= len(df_pp[df_pp[1]!='I-ARGM-MNR'])/len(df_pp)*100
df_pp.to_csv("../results/pp_attachment/pp_attachment_arg-mnr_bilstm.csv")

In [13]:
list_pp_attachment_arg1 = make_data_pp_attachment_arg1(nsamples)

df_pp_arg1 = pd.DataFrame(list_pp_attachment_arg1)
df_pp_arg1['pp_tag'] = df_pp_arg1[0].apply(get_pp_tags)
df_pp_arg1['pp_descr'] = df_pp_arg1[0].apply(get_pp_descr)

df_pp_arg1[1] = df_pp_arg1['pp_tag'].apply(find_final_min_one_arg)

In [14]:
error_rate_arg1_bilstm = len(df_pp_arg1[df_pp_arg1[1] != 'I-ARG1'])/len(df_pp_arg1)
df_pp_arg1.to_csv("../results/pp_attachment/pp_attachment_arg1_bilstm.csv")

In [15]:
print("bilstm")
print("argm-mnr: ",error_rate_arg_mnr_bilstm)
print("arg1",error_rate_arg1_bilstm )
print()
print("BERT")
print("argm-mnr: ",error_rate_arg_mnr_bert)
print("arg1",error_rate_arg1_bert )

bilstm
argm-mnr:  81.33333333333333
arg1 0.0

BERT
argm-mnr:  16.666666666666664
arg1 0.0


In [19]:
pd.set_option('max_colwidth',1000)
df_pp[df_pp[1]!='I-ARGM-MNR']

,0,pp_tag,pp_descr,1
0,I saw the rolling office chair with my eyes.,"[B-ARG0, B-V, B-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, O]",[ARG0: I] [V: saw] [ARG1: the rolling office chair with my eyes] .,I-ARG1
1,I saw the orange juice box with my eyes.,"[B-ARG0, B-V, B-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, O]",[ARG0: I] [V: saw] [ARG1: the orange juice box with my eyes] .,I-ARG1
2,I saw the telephone with my eyes.,"[B-ARG0, B-V, B-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, O]",[ARG0: I] [V: saw] [ARG1: the telephone with my eyes] .,I-ARG1
3,I saw the toothbrush with my eyes.,"[B-ARG0, B-V, B-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, O]",[ARG0: I] [V: saw] [ARG1: the toothbrush with my eyes] .,I-ARG1
4,I saw the footstool with my eyes.,"[B-ARG0, B-V, B-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, O]",[ARG0: I] [V: saw] [ARG1: the footstool with my eyes] .,I-ARG1
...,...,...,...,...
365,I tasted the soap dispenser with my tongue.,"[B-ARG0, B-V, B-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, O]",[ARG0: I] [V: tasted] [ARG1: the soap dispenser with my tongue] .,I-ARG1
375,I tasted the spoon with my tongue.,"[B-ARG0, B-V, B-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, O]",[ARG0: I] [V: tasted] [ARG1: the spoon with my tongue] .,I-ARG1
391,I tasted the coffee table with my tongue.,"[B-ARG0, B-V, B-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, O]",[ARG0: I] [V: tasted] [ARG1: the coffee table with my tongue] .,I-ARG1
393,I tasted the sofa with my tongue.,"[B-ARG0, B-V, B-ARG1, I-ARG1, B-ARG2, I-ARG2, I-ARG2, O]",[ARG0: I] [V: tasted] [ARG1: the sofa] [ARG2: with my tongue] .,I-ARG2


In [21]:
len(df_pp)

150

In [22]:
df_pp

,0,pp_tag,pp_descr,1
0,I saw the rolling office chair with my eyes.,"[B-ARG0, B-V, B-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, O]",[ARG0: I] [V: saw] [ARG1: the rolling office chair with my eyes] .,I-ARG1
1,I saw the orange juice box with my eyes.,"[B-ARG0, B-V, B-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, O]",[ARG0: I] [V: saw] [ARG1: the orange juice box with my eyes] .,I-ARG1
2,I saw the telephone with my eyes.,"[B-ARG0, B-V, B-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, O]",[ARG0: I] [V: saw] [ARG1: the telephone with my eyes] .,I-ARG1
3,I saw the toothbrush with my eyes.,"[B-ARG0, B-V, B-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, O]",[ARG0: I] [V: saw] [ARG1: the toothbrush with my eyes] .,I-ARG1
4,I saw the footstool with my eyes.,"[B-ARG0, B-V, B-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, O]",[ARG0: I] [V: saw] [ARG1: the footstool with my eyes] .,I-ARG1
...,...,...,...,...
373,I tasted the fork with my tongue.,"[B-ARG0, B-V, B-ARG1, I-ARG1, B-ARGM-MNR, I-ARGM-MNR, I-ARGM-MNR, O]",[ARG0: I] [V: tasted] [ARG1: the fork] [ARGM-MNR: with my tongue] .,I-ARGM-MNR
375,I tasted the spoon with my tongue.,"[B-ARG0, B-V, B-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, O]",[ARG0: I] [V: tasted] [ARG1: the spoon with my tongue] .,I-ARG1
391,I tasted the coffee table with my tongue.,"[B-ARG0, B-V, B-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, I-ARG1, O]",[ARG0: I] [V: tasted] [ARG1: the coffee table with my tongue] .,I-ARG1
393,I tasted the sofa with my tongue.,"[B-ARG0, B-V, B-ARG1, I-ARG1, B-ARG2, I-ARG2, I-ARG2, O]",[ARG0: I] [V: tasted] [ARG1: the sofa] [ARG2: with my tongue] .,I-ARG2
